# Baby Names by state

Using the SSA state babynames data from https://www.ssa.gov/oact//babynames/limits.html

In [88]:
import pandas as pd
import numpy as np
import plotly.express as px
import glob
from dash import Dash, html, dcc, callback, Output, Input

In [3]:
# read in the state data

# textfiles
txt_files = glob.glob(".\\babynamesData\\*.txt")
  
# Empty dataframe
combined_df = pd.DataFrame()

for txt_file in txt_files:
    df = pd.read_table(txt_file,delimiter=',',names=['State','Sex','Year','Name','Number'])
    combined_df = pd.concat([combined_df,df])
combined_df.head()

,State,Sex,Year,Name,Number
0,AK,F,1910,Mary,14
1,AK,F,1910,Annie,12
2,AK,F,1910,Anna,10
3,AK,F,1910,Margaret,8
4,AK,F,1910,Helen,7


In [90]:
name = "Ashton"
year_start = 2000
year_end = 2022
sex_fm = None
    
app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(["","M","F"], "", id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)

def update_graph(value):
        # get last 3 years, and a specific name
    if sex_fm is None:
        df_all = combined_df
        col = 'mint'
        baby_str = "Babies"
    elif sex_fm == "F":
        df_all = combined_df.loc[(combined_df['Sex']==sex_fm)] 
        col = 'purpor'
        baby_str = "Female Babies"
    elif sex_fm == "M":
        df_all = combined_df.loc[(combined_df['Sex']==sex_fm)] 
        col = 'blues'
        baby_str = "Male Babies"

    if year_start<year_end:
        title_str=f'Percent of {baby_str} Named {name} by State from {year_start} to {year_end}, x1000'
    elif year_start>=year_end:
        year_end=year_start
        title_str=f'Percent of {baby_str} Named {name} by State in {year_start}, x1000'
        
    df_all = df_all.loc[(df_all['Year']>=year_start)&(df_all['Year']<=year_end)]
    df_sub = df_all.loc[(df_all['Name']==name)] 

    df_sub=df_sub.drop(['Year'],axis=1).groupby(by='State').sum(numeric_only=True)
    df_all=df_all.drop(['Year'],axis=1).groupby(by='State').sum(numeric_only=True)
    df_all.rename(columns={"Number":"Total Number"},inplace=True)
    df_sub=pd.concat([df_sub,df_all],axis=1)
    df_sub['Percent x1000'] = df_sub["Number"]/df_sub["Total Number"]*1000
    df_sub = df[df.country==value]
    fig = px.choropleth(df_sub,
                    locations=df_sub.index,
                    locationmode="USA-states",
                    scope="usa",
                    color='Percent x1000',
                    color_continuous_scale=col
                   )
    # center the title
    fig.update_layout(title_text=title_str, title_x=0.5)
    fig.show() 
    return px.line(dff, x='year', y='pop')


# df_sub.head()
# plot a choropleth with color range by count per state
fig = px.choropleth(df_sub,
                    locations=df_sub.index,
                    locationmode="USA-states",
                    scope="usa",
                    color='Percent x1000',
                    color_continuous_scale=col
                   )
# center the title
fig.update_layout(title_text=title_str, title_x=0.5)
fig.show() 

# ['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
#              'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
#              'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
#              'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
#              'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
#              'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
#              'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
#              'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
#              'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
#              'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
#              'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
#              'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
#              'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
#              'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
#              'ylorrd']